<a href="https://colab.research.google.com/github/RanulRathnayake/Final-Year-Research-/blob/main/XML_RoBERTa(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaModel, Trainer, TrainingArguments
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DataSets/FAQs dataset.csv')

print(df.head())
print(df.columns)


                                              text             entity  \
0               Mata tax return file karanna oney.  income_tax_filing   
1            Mata tax return submit karanna oneda?  income_tax_filing   
2      Income tax return danna widiyak kiyanawada?  income_tax_filing   
3             Tax return submit karanna puluwanda?  income_tax_filing   
4  Mata income tax return ekak file karanna oneda?  income_tax_filing   

                    intent  
0          file_tax_return  
1        submit_tax_return  
2   how_to_file_tax_return  
3    can_submit_tax_return  
4  need_to_file_tax_return  
Index(['text', ' entity', ' intent'], dtype='object')


In [ ]:
entity_encoder = LabelEncoder()
intent_encoder = LabelEncoder()

df['entity_label'] = entity_encoder.fit_transform(df[' entity'])
df['intent_label'] = intent_encoder.fit_transform(df[' intent'])

In [ ]:
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
train_texts, val_texts, train_entity_labels, val_entity_labels, train_intent_labels, val_intent_labels = train_test_split(
    df['text'].tolist(),
    df['entity_label'].tolist(),
    df['intent_label'].tolist(),
    test_size=0.2,
    random_state=42
)

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [ ]:
class EntityIntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, entity_labels, intent_labels):
        self.encodings = encodings
        self.entity_labels = entity_labels
        self.intent_labels = intent_labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['entity_labels'] = torch.tensor(self.entity_labels[idx])
        item['intent_labels'] = torch.tensor(self.intent_labels[idx])
        return item

    def __len__(self):
        return len(self.entity_labels)

In [ ]:
train_dataset = EntityIntentDataset(train_encodings, train_entity_labels, train_intent_labels)
val_dataset = EntityIntentDataset(val_encodings, val_entity_labels, val_intent_labels)


In [ ]:
class XLMRobertaForMultiTaskClassification(torch.nn.Module):
    def __init__(self, model_name, num_entity_labels, num_intent_labels):
        super(XLMRobertaForMultiTaskClassification, self).__init__()
        self.roberta = XLMRobertaModel.from_pretrained(model_name)
        self.entity_classifier = torch.nn.Linear(self.roberta.config.hidden_size, num_entity_labels)
        self.intent_classifier = torch.nn.Linear(self.roberta.config.hidden_size, num_intent_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, entity_labels=None, intent_labels=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]
        pooled_output = outputs[1]

        entity_logits = self.entity_classifier(sequence_output)
        intent_logits = self.intent_classifier(pooled_output)

        loss = 0
        if entity_labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss += loss_fct(entity_logits.view(-1, self.entity_classifier.out_features), entity_labels.view(-1,1).squeeze(1))
        if intent_labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss += loss_fct(intent_logits.view(-1, self.intent_classifier.out_features), intent_labels.view(-1).squeeze(1))

        return loss, entity_logits, intent_logits

model = XLMRobertaForMultiTaskClassification('xlm-roberta-base', len(entity_encoder.classes_), len(intent_encoder.classes_))

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

ValueError: Expected input batch_size (248) to match target batch_size (8).